<a href="https://colab.research.google.com/github/WesselBoi/Suicidal-Content-detection/blob/main/infer_logit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pickle

In [ ]:
try:
    os.chdir("/content/drive/MyDrive/SuicideModel")
    print("Directory changed")
except OSError:
    print("Error: Can't change the Current Working Directory")


Directory changed


In [ ]:
def load_doc(filename):
    with open(filename, 'r') as f:
        text = f.read()
    return text

vocab = set(load_doc('Data/vocab.txt').split())

def load_embedding(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()[1:]
    embedding = dict()
    for line in lines:
        parts = line.split()
        embedding[parts[0]] = np.asarray(parts[1:], dtype='float32')
    return embedding

word2vec = load_embedding('Data/embedding_word2vec.txt')

In [ ]:
with open('models_logit_word2vec.pkl', 'rb') as f:
    logit_model = pickle.load(f)


In [ ]:
print(logit_model.n_features_in_)  # should be 300 for Word2Vec-trained

300


In [ ]:
def clean_line(line, vocab):
    if not isinstance(line, str):
        line = ""
    tokens = line.split()
    return [w for w in tokens if w in vocab]

def document_vector(doc, embeddings):
    tokens = [word for word in doc if word in embeddings]
    if not tokens:
        return np.zeros(next(iter(embeddings.values())).shape)
    vectors = [embeddings[word] for word in tokens]
    return np.mean(vectors, axis=0)


In [ ]:
def check_intent_logit(text):
    tokens = clean_line(text, vocab)
    vec = document_vector(tokens, word2vec).reshape(1, -1)
    pred = logit_model.predict(vec)[0]
    prob = logit_model.predict_proba(vec)[0][1]
    return pred, prob


In [ ]:
print(check_intent_logit("How are you?"))  # Non-suicidal example
print(check_intent_logit("I don't belong to this world"))  # Suicidal example

(np.int64(0), np.float64(0.14377364653671426))
(np.int64(1), np.float64(0.949112878193776))


In [ ]:
print(check_intent_logit("living is overrated"))

(np.int64(1), np.float64(0.9168214388702229))


In [ ]:
print(check_intent_logit("Life goes on, and I just keep moving."))

(np.int64(1), np.float64(0.9458033365752366))
